In [1]:
!pip install transformers datasets seqeval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 3.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=d67526b77a0f4c9d0334199659ed1de71899a1b04712c4281329bac357802967
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resol

1. Load Pre-Trained **Model**

In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification

MODEL_NAME = "xlm-roberta-base"  # Replace with "bert-tiny-amharic" or another Amharic-supporting model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=4)  # Adjust `num_labels` based on your dataset


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2. Load and Parse Labeled Dataset

In [4]:
import pandas as pd

def parse_conll_to_dataframe(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        tokens, labels = [], []
        for line in f:
            line = line.strip()
            if not line:  # Blank line indicates a new sentence
                if tokens:
                    data.append((tokens, labels))
                    tokens, labels = [], []
            else:
                token, label = line.split("\t")
                tokens.append(token)
                labels.append(label)
        if tokens:  # Append last sentence
            data.append((tokens, labels))
    return pd.DataFrame(data, columns=["tokens", "labels"])

# Load the dataset
df = parse_conll_to_dataframe("/content/drive/MyDrive/week 5/labeled_data.conll")


3. Tokenize Data and Align Labels

In [16]:
from transformers import DataCollatorForTokenClassification

# Create a label mapping (string to integer)
unique_labels = set()  # Collect all unique labels
for labels in df["labels"]:
    unique_labels.update(labels)

label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}  # Reverse mapping for later use

def tokenize_and_align_labels(batch):
    tokenized_inputs = tokenizer(batch["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(batch["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_id = None
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)  # Special tokens
            elif word_id != previous_word_id:
                # Convert label to integer using the mapping
                label_ids.append(label2id[label[word_id]])
            else:
                label_ids.append(-100)  # Subword tokens
            previous_word_id = word_id
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Convert DataFrame to Hugging Face Dataset
from datasets import Dataset

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

print("Label Mapping (label2id):", label2id)  # Print for inspection
print("Reverse Mapping (id2label):", id2label)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Label Mapping (label2id): {'O': 0, 'I-PRICE': 1, 'B-PRICE': 2, 'B-LOC': 3, 'B-Product': 4}
Reverse Mapping (id2label): {0: 'O', 1: 'I-PRICE', 2: 'B-PRICE', 3: 'B-LOC', 4: 'B-Product'}


4. Tokenize Data and Align Labels

In [17]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,
    load_best_model_at_end=True
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ValueError: --load_best_model_at_end requires the save and eval strategy to match, but found
- Evaluation strategy: IntervalStrategy.EPOCH
- Save strategy: SaveStrategy.STEPS